## Understanding Run Decision Tree1 

In [1]:
# Import packages
import shutil
from pathlib import Path

import pandas as pd

from apply.apply_asset_level_hotspot import apply_deforestation_hotspots_assets
from apply.apply_controversy import apply_controversy_filters
from apply.apply_decision_tree1 import apply_dt1_conservative_approach, apply_dt1_weighted_average_approach
from apply.apply_direct_attribution import apply_direct_attribution_filter
from apply.apply_forest_and_finance import apply_forest_and_finance
from apply.apply_number_of_pairs import apply_number_of_pairs
from apply.apply_sectoral_filter_nace import apply_sectoral_filters_nace
from apply.apply_supply_chain_filter import apply_supply_chain_filter

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Loading the preliminary codes of the python file

In [2]:
# Import functions from other scripts
from filepaths import PATH_TO_PORTFOLIO_DATA, PATH_TO_COMPANY_SPECIFIC_ASSET_LEVEL_DATA, PATH_TO_OUTPUT_FOLDER, \
    PATH_TO_INPUT_FOLDER

In [3]:
from generate.generate_combine_asset_data import combine_asset_datasets
from prep.prep_asset_level_merge import merge_asset_level_to_portfolio_companies
from prep.prep_weighted_country_sector_pairs import prep_weighted_country_sector_pairs

In [5]:
from user_input import (
    # GENERAL SETTINGS
    exclude_financial_institutions,
    perform_full_analysis,
    bias_towards_existing_data,
    bias_towards_missing_data,
    equal_information_importance,
    climate_and_company_information_importance,
    specific_information_importance,
    importance_revenue_info,
    importance_hierarchy_info,
    importance_asset_info,
    importance_headquarter_info,
    # EXECUTE DT1
    use_io_model_score,
    use_trase_flag,
    use_flag_direct,
    use_flag_indirect,
    use_flag_forest500,
    recent_controversies_cutoffs,  # if true, absolute cutoffs are used
    historical_controversies_cutoffs,  # suggest to keep this false

    # EXECUTE BUCKETING
    flag_direct_threshold_high,
    flag_direct_threshold_medium,
    flag_indirect_threshold_high,
    flag_indirect_threshold_medium,
    IO_threshold_high,
    IO_threshold_medium,
    recent_controversies_threshold_high,
    recent_controversies_threshold_medium,
    historical_controversies_threshold_high,
    historical_controversies_threshold_medium,
    cutoff_direct_attribution,
    subsidiary_data_exists,
    hotspot_assets_threshold,
    hotspot_subsidiaries_threshold,

    # OVERLAY WITH HOTSPOTS
    DISTANCE_THRESHOLD_ASSETS
)

In [6]:
from utils import clean_df_portfolio

# from apply.apply_sectoral_filter_gics import apply_sectoral_filters_gics # not yet incorporated

### ---------------------------------------------------###
### 0) CORE DATA INPUT & SET PARAMETERS                ###
### ---------------------------------------------------###



In [7]:
"""
0.1: QUALITY CHECK PARAMETERS
"""
# Define lists of variables to check
boolean_variables = [
    exclude_financial_institutions, perform_full_analysis, bias_towards_existing_data,
    equal_information_importance, climate_and_company_information_importance, specific_information_importance,
    use_io_model_score, use_trase_flag, use_flag_direct, use_flag_indirect, use_flag_forest500,
    recent_controversies_cutoffs, historical_controversies_cutoffs
]

numerical_variables = [
    flag_direct_threshold_high, flag_direct_threshold_medium, flag_indirect_threshold_high,
    flag_indirect_threshold_medium, cutoff_direct_attribution, IO_threshold_high, IO_threshold_medium,
    recent_controversies_threshold_high, recent_controversies_threshold_medium,
    historical_controversies_threshold_high, historical_controversies_threshold_medium,
    hotspot_assets_threshold, hotspot_subsidiaries_threshold, importance_revenue_info, importance_hierarchy_info,
    importance_asset_info, importance_headquarter_info
]


In [8]:
# Perform boolean check
if any(not isinstance(variable, bool) for variable in boolean_variables):
    raise ValueError("All of the user inputs listed above must be boolean values (True or False), please check.")

# Perform numerical check
if any(not isinstance(variable, (int, float)) for variable in numerical_variables):
    raise ValueError("All of the user inputs listed above must be numerical values (integers or floats), please check.")

# Perform bias check
if bias_towards_existing_data == bias_towards_missing_data:
    raise ValueError('Please check your chosen bias and decide for one of them')

# Perform weighting check
if sum([
    equal_information_importance,
    climate_and_company_information_importance,
    specific_information_importance
]) >= 2:
    raise ValueError("Two or more weightings are set to True.")

Done till 0.1 and now we start with 0.2 ...

In [9]:
"""
0.2: Delete interim files that were stored
"""

# If perform_full_analysis is set to True, delete the output folder and create a new one

if perform_full_analysis:
    shutil.rmtree(PATH_TO_OUTPUT_FOLDER, ignore_errors=True)
    PATH_TO_OUTPUT_FOLDER.mkdir()
    Path(PATH_TO_OUTPUT_FOLDER / 'internal_data').mkdir()

In [11]:
df_portfolio = pd.read_excel(PATH_TO_PORTFOLIO_DATA, sheet_name='data').astype(str)

print(df_portfolio.head())


In [14]:
print(df_portfolio.head())

                     name      permid                   lei ticker      ric  \
0     Erste Group Bank AG  8589934296  PQOH26KWDF7CG10L6792    EBS  ERST.VI   
1              Verbund AG  4295859071  5299006UDSEJCTTEJS30    VER  VERB.VI   
2  Aristocrat Leisure Ltd  4295857388  529900TE2HO3OZKYKC27    ALL   ALL.AX   
3  ANZ Group Holdings Ltd  5083687566  9845005D9C6BAA10BK72    ANZ   ANZ.AX   
4    Aurizon Holdings Ltd  4296075611  5299005PC1O6NOWL3H48    AZJ   AZJ.AX   

  ipo_year                               headquarters_address  \
0   1997.0        Am Belvedere 1\nWIEN\nWIEN\n1100\nAustria\n   
1   1988.0             Am Hof 6a\nWIEN\nWIEN\n1010\nAustria\n   
2   1996.0  Bldg a\nPinnacle\n85 Epping Rd\nNORTH RYDE\nNE...   
3   1969.0                        VICTORIA\n3008\nAustralia\n   
4   2010.0  Aurizon\n900 Ann Street\nFORTITUDE VALLEY\nQUE...   

                                  registered_address trbc_code_lev3  \
0        Am Belvedere 1\nWIEN\nWIEN\n1100\nAustria\n       5510

Done till 0.2: Deleting interim files that were stored and start with 0.3: LOAD DATA


In [10]:
"""
0.3: LOAD DATA
"""

# Load df_portfolio (by default: MSCI ACWI Index with limited data)
df_portfolio = pd.read_excel(PATH_TO_PORTFOLIO_DATA, sheet_name='data').astype(str)
df_portfolio = clean_df_portfolio(df_portfolio)  # apply clean_df_portfolio from utils
df_portfolio["identifier"] = df_portfolio["permid"].astype(str)  # define identifier column

# Load asset level data (by default: data from SFI, CLimate Trace, GEM)
if Path(PATH_TO_COMPANY_SPECIFIC_ASSET_LEVEL_DATA).exists():
    asset_data_columns = ['permid', 'nace_code', 'country_iso', 'final_weight']
    df_asset_matches_aggregated = pd.read_csv(PATH_TO_COMPANY_SPECIFIC_ASSET_LEVEL_DATA,
                                              usecols=asset_data_columns,
                                              dtype={'permid': str, 'nace_code': str, 'country_iso': str,
                                                     'final_weight': float})
else:
    # Load the >70k assets from the different sources
    df_asset_data_raw = combine_asset_datasets()

    # Map to portfolio companies via identifiers & text matching
    df_asset_matches_disaggregated, df_asset_matches_aggregated = merge_asset_level_to_portfolio_companies(df_portfolio,
                                                                                                           df_asset_data_raw)

# Define main identifier column for asset data
df_asset_matches_aggregated["identifier"] = df_asset_matches_aggregated["permid"].astype(str)

# Derive country-sector pairs for each company in the portfolio
df_portfolio_country_sector_pairs = prep_weighted_country_sector_pairs(df_asset_matches_aggregated,
                                                                       df_portfolio,
                                                                       bias_towards_existing_data,
                                                                       bias_towards_missing_data,
                                                                       importance_asset_info,
                                                                       importance_headquarter_info)

### --------------------------------------------------------###
### 1) COLLECT FLAGS FOR DECISION TREE 1 (STEP1 of report)  ###
### --------------------------------------------------------###

Dataset has been cleaned, ready for further processing.
Processing and combining asset-level datasets...
Processing data from Global Energy Monitor (GEM)...
Processing data from Climate Trace...
Processing data from Spatial Finance Initiative (SFI)...
Asset-level datasets processed and combined!
Merging asset-level data with portfolio company data...
Mapping based on identifiers...
Mapping via direct text matching...
Mapping via fuzzy string matching...
matched_permid_fuzzy_parent_name has been added to df_asset with a score cutoff of 95.
matched_permid_fuzzy_owner_name has been added to df_asset with a score cutoff of 95.
matched_permid_fuzzy_operator_name has been added to df_asset with a score cutoff of 95.
Number of matches in 'matched_permid_identifier': 4722
Number of matches in 'matched_permid_direct_text': 10097
Number of matches in 'matched_permid_fuzzy': 326
Total number of matches in consolidated 'matched_permid': 11255
Removing duplicates...
Removed 1485 rows out of 11255 t

In [16]:
# Understanding the above code

df_portfolio = pd.read_excel(PATH_TO_PORTFOLIO_DATA, sheet_name='data').astype(str)

df_portfolio.head()

,name,permid,lei,ticker,ric,ipo_year,headquarters_address,registered_address,trbc_code_lev3,trbc_label_lev3,trbc_comment_lev3,country_name,country_iso,country_name_incorporation,country_iso_incorporation,nace_code,nace_desc
0,Erste Group Bank AG,8589934296,PQOH26KWDF7CG10L6792,EBS,ERST.VI,1997.0,Am Belvedere 1\nWIEN\nWIEN\n1100\nAustria\n,Am Belvedere 1\nWIEN\nWIEN\n1100\nAustria\n,551010.0,Banking Services,"Providers of retail and commercial banking, co...",Austria,AT,Austria,AT,64.19,Other monetary intermediation
1,Verbund AG,4295859071,5299006UDSEJCTTEJS30,VER,VERB.VI,1988.0,Am Hof 6a\nWIEN\nWIEN\n1010\nAustria\n,Am Hof 6 A\nWIEN\nWIEN\n1010\nAustria\n,591010.0,Electric Utilities & IPPs,Generators and distributors of electric power....,Austria,AT,Austria,AT,35.11,Production of electricity
2,Aristocrat Leisure Ltd,4295857388,529900TE2HO3OZKYKC27,ALL,ALL.AX,1996.0,Bldg a\nPinnacle\n85 Epping Rd\nNORTH RYDE\nNE...,"Building A, Pinnacle Office Park\n85 Epping Ro...",533010.0,Hotels & Entertainment Services,"Operators of hotels, motels, cruise lines, res...",Australia,AU,Australia,AU,32.4,Manufacture of games and toys
3,ANZ Group Holdings Ltd,5083687566,9845005D9C6BAA10BK72,ANZ,ANZ.AX,1969.0,VICTORIA\n3008\nAustralia\n,Docklands\nVICTORIA\n3008\nAustralia\n,551010.0,Banking Services,"Providers of retail and commercial banking, co...",Australia,AU,Australia,AU,64.19,Other monetary intermediation
4,Aurizon Holdings Ltd,4296075611,5299005PC1O6NOWL3H48,AZJ,AZJ.AX,2010.0,Aurizon\n900 Ann Street\nFORTITUDE VALLEY\nQUE...,"Level 8, 900 Ann Street\nFORTITUDE VALLEY\nQUE...",524050.0,"['Freight&Logistics Services', 'Freight & Logi...",Providers of freight and logistics services by...,Australia,AU,Australia,AU,49.2,Freight rail transport


In [ ]:
# Understanding the above code part 2
df_portfolio = clean_df_portfolio(df_portfolio)  # apply clean_df_portfolio from utils
df_portfolio["identifier"] = df_portfolio["permid"].astype(str)  # define identifier column

df_portfolio.head()

Understaing the above main code part 3. 

1. df_portfolio cleaned using the function clean_df_portfolio
    a. weird .0 at the end is removed
    b. HK companies with nan designated with HK, because they are the HK companies. What is HK companies though? check it out
    c. jersey is part of the UK, so changed to UK
2. Identifier has been designated. lei ~identifier

Lastly head prints the cleaned data, which is quite neat.


THIS WAS ALL FOR PORTFOLIO DATA.

TIME TO LOAD ASSET DATA AND PROCESS FOR PREPARATION

In [ ]:
# Load asset level data (by default: data from SFI, CLimate Trace, GEM)
if Path(PATH_TO_COMPANY_SPECIFIC_ASSET_LEVEL_DATA).exists():
    asset_data_columns = ['permid', 'nace_code', 'country_iso', 'final_weight']
    df_asset_matches_aggregated = pd.read_csv(PATH_TO_COMPANY_SPECIFIC_ASSET_LEVEL_DATA,
                                              usecols=asset_data_columns,
                                              dtype={'permid': str, 'nace_code': str, 'country_iso': str,
                                                     'final_weight': float})
else:
    # Load the >70k assets from the different sources
    df_asset_data_raw = combine_asset_datasets()

    # Map to portfolio companies via identifiers & text matching
    df_asset_matches_disaggregated, df_asset_matches_aggregated = merge_asset_level_to_portfolio_companies(df_portfolio,
                                                                                                           df_asset_data_raw)